# Extracting Misclassified Examples from Averaged Perceptron

trid to get:
15 examples where the model predicted **PASS** but actual was **FAIL** (False Positives)
10 example where the model predicted **FAIL** but actual was **PASS** (False Negatives)

Used davids avf pperceptron 

## 1. Imports and Setup

In [2]:
import random
import torch
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
seed = 1234

if seed is not None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

print("done runnin")

done runnin


## 2. Data Cleaning (from Data_Cleaning_Code.ipynb)

In [3]:
education_data = pd.read_csv('students_clean.csv')


education_data.drop('Parent_Education_Level', axis=1, inplace=True) 

education_data['Gender'] = education_data['Gender'].replace({'Male': 1, 'Female': 0}).astype(int)
education_data['Internet_Access_at_Home'] = education_data['Internet_Access_at_Home'].replace({'Yes': 1, 'No': 0}).astype(int)
education_data['Extracurricular_Activities'] = education_data['Extracurricular_Activities'].replace({'Yes': 1, 'No': 0}).astype(int)


# Low = 1, Medium = 2, High = 3
mapper = {'low': 1, 'medium': 2, 'high': 3}

education_data['Family_Income_Level'] = (
    education_data['Family_Income_Level']
      .astype(str)                  # works even if the value is already 1/2/3 or NaN
      .str.strip().str.lower()
      .map(mapper)                  # returns NaN where no mapping found
      .fillna(education_data['Family_Income_Level'])  # keepin the original numeric/blank entries
      .astype('Int64')              #  nullable integer dtype
)

labels = open('departments.txt').read().splitlines()
department_mapping = {name: index for index, name in enumerate(labels)}
department_indices = education_data['Department'].map(department_mapping)
education_data.insert(3, 'department index', department_indices)

mapper = {'A': 1, 'B': 1, 'C': 1, 'D':0,'F':0}

education_data['Grade'] = (
    education_data['Grade']
      .astype(str)              # convert everything to string
      .str.strip().str.upper()  # remove spaces and standardize to uppercase
      .map(mapper)              # map letters to numbers
)

education_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'students_clean.csv'

## 3. Dataset Class and Perceptron (from our codebase)

In [73]:
#  class from perceptron notebook
class MyDataset(Dataset):
    def __init__(self, df, feature_cols, target_col):
        self.df = df
        self.feature_cols = feature_cols
        self.target_col = target_col

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        x = torch.tensor(row[self.feature_cols].to_numpy(dtype=np.float32), dtype=torch.float32)
        y = torch.tensor(row[self.target_col], dtype=torch.long)
        return x, y

In [74]:
def train_perceptron(train_dl, n_features, pos_class):
    # First initialize the model.
    w = np.zeros(n_features)
    b = 0
    n_errors = 0
    weight_steps = []
    total_pos_in_train = 0

    # Adding this in for debug purposes to track the changes to the weight vectors on each
    # round.
    
    # Average perceptron features
    totalW = np.zeros(n_features)
    totalB = 0;
    updateCount = 0;
    
    # Now loop through each batch.
    for batch_idx, (x, y) in tqdm(enumerate(train_dl), total=len(train_dl),):
        
        x_curr_np = x.numpy()
        y_curr_np = y.numpy()

        total_pos_in_train += (y_curr_np == 1).sum(axis=0)
        

        # Now perform the training/classification loop.
        scores = x_curr_np @ w + b
       
        
        y_pred = (scores > 0).astype(int)


        # Now we vectorize the update to make this more efficient.
        pred_error = y_curr_np - y_pred
        n_errors += np.sum(np.abs(pred_error) != 0) # If the pred error is zero then it is correct.

        # First append the previous weights to weight steps which will be used for debuging puprposes.
        weight_steps.append((pred_error[:,None]*x_curr_np).sum(axis=0).copy())
        
        w += (pred_error[:,None]*x_curr_np).sum(axis=0) # Re-shape pred errors to update and only add
                                                        # inccorect preds, axis=0 for rows.
        b += pred_error.sum()

        # Now print out the weights and bias updates every update if we are in debug mode.
        

    # Now once we are done training the result is the weights and biases.
    return (w,b,n_errors,weight_steps.copy(),total_pos_in_train) # I am just copying to avoid weird cases due to mutability of list.

## 4. Create Train/Dev/Test Splits

In [75]:
# Split using same random state as our PassFail notebook
train_df, test_df = train_test_split(education_data, train_size=0.9, random_state=seed)
train_df, dev_df = train_test_split(train_df, train_size=0.8, random_state=seed)

train_df.reset_index(inplace=True, drop=True)
dev_df.reset_index(inplace=True, drop=True)
test_df.reset_index(inplace=True, drop=True)

print(f'Train rows: {len(train_df):,}')
print(f'Dev rows: {len(dev_df):,}')
print(f'Test rows: {len(test_df):,}')

print(f"\nDev set Pass/Fail distribution:")
print(f"  Pass (1): {(dev_df['Grade'] == 1).sum()}")
print(f"  Fail (0): {(dev_df['Grade'] == 0).sum()}")

Train rows: 3,600
Dev rows: 900
Test rows: 500

Dev set Pass/Fail distribution:
  Pass (1): 555
  Fail (0): 345


## 5. Train Averaged Perceptron (10 models)

In [80]:
#using After Final features for most comprehensive results)
n_perceptrons = 10
weight_vecs = []
bias_vecs = []


features_lst = ['Attendance (%)', 'Extracurricular_Activities', 'Midterm_Score', 
                'Final_Score', 'Assignments_Avg', 'Quizzes_Avg', 
                'Participation_Score', 'Projects_Score', 
                'Stress_Level (1-10)', 'Sleep_Hours_per_Night']
num_feat = len(features_lst)
batch_size = 64
shuffle = True

print(f"Training {n_perceptrons} perceptrons...\n")

for i in range(n_perceptrons):
    print(f"Training perceptron {i+1}/{n_perceptrons}")
    train_ds = MyDataset(train_df, features_lst, 'Grade')
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle)
    
    w_curr, b_curr, error_curr, weight_hist_curr, tot_train_pos_curr = train_perceptron(train_dl, num_feat, pos_class=1)
    
    weight_vecs.append(w_curr)
    bias_vecs.append(b_curr)

# Average the weights and biases
w_avg = np.mean(weight_vecs, axis=0)
b_avg = np.mean(bias_vecs, axis=0)

print(f"\n{'='*70}")
print("Averaged Perceptron Training Complete")
print(f"{'='*70}")
print(f"Averaged weight vector shape: {w_avg.shape}")
print(f"Averaged bias value: {b_avg:.4f}")

Training 10 perceptrons...

Training perceptron 1/10


  0%|          | 0/57 [00:00<?, ?it/s]

Training perceptron 2/10


  0%|          | 0/57 [00:00<?, ?it/s]

Training perceptron 3/10


...

Training perceptron 9/10


  0%|          | 0/57 [00:00<?, ?it/s]

Training perceptron 10/10


  0%|          | 0/57 [00:00<?, ?it/s]


Averaged Perceptron Training Complete
Averaged weight vector shape: (10,)
Averaged bias value: -97.3000


## 6. Evaluate on Dev Set and Get Predictions

In [81]:
# Make predictions on dev set
X_dev = dev_df[features_lst].to_numpy()
dev_y_true = dev_df['Grade'].to_numpy()
dev_y_pred = ((X_dev @ w_avg + b_avg) > 0).astype(int)

# accuracy
n_correct_dev = (dev_y_true == dev_y_pred).sum()
accuracy = (n_correct_dev / dev_y_true.shape[0]) * 100

print(f"Dev Set Performance:")
print(f"  Accuracy: {accuracy:.2f}%")
print(f"  Correct predictions: {n_correct_dev}/{len(dev_y_true)}")
print(f"\nPrediction Distribution:")
print(f"  Predicted Pass (1): {(dev_y_pred == 1).sum()}")
print(f"  Predicted Fail (0): {(dev_y_pred == 0).sum()}")

Dev Set Performance:
  Accuracy: 69.33%
  Correct predictions: 624/900

Prediction Distribution:
  Predicted Pass (1): 795
  Predicted Fail (0): 105


## 7. Extract Misclassified Examples
False Positives (FP): Predicted PASS but actual was FAIL (15 examples)
False Negatives (FN): Predicted FAIL but actual was PASS (10 examples)

In [64]:
# extract misclassifications
# False Positives: Predicted Pass (1), Actual Fail (0)
false_positives_mask = (dev_y_pred == 1) & (dev_y_true == 0)
false_positives_indices = np.where(false_positives_mask)[0]

# False Negatives: Predicted Fail (0), Actual Pass (1)
false_negatives_mask = (dev_y_pred == 0) & (dev_y_true == 1)
false_negatives_indices = np.where(false_negatives_mask)[0]

print(f"{'='*70}")
print("Misclassification Summary")
print(f"{'='*70}")
print(f"False Positives (predicted Pass, actual Fail): {len(false_positives_indices)}")
print(f"False Negatives (predicted Fail, actual Pass): {len(false_negatives_indices)}")
print(f"\nExtracting:")
print(f"  - 15 False Positives")
print(f"  - 10 False Negatives")

Misclassification Summary
False Positives (predicted Pass, actual Fail): 48
False Negatives (predicted Fail, actual Pass): 323

Extracting:
  - 15 False Positives
  - 10 False Negatives


In [82]:
# xtract 15 False Positives
num_fp_to_extract = min(15, len(false_positives_indices))
fp_indices = false_positives_indices[:num_fp_to_extract]

#  DataFrame with all relevant information
false_positives_df = dev_df.iloc[fp_indices].copy()
false_positives_df['Predicted_Grade'] = 1  # Pass
false_positives_df['Actual_Grade'] = 0     # Fail
false_positives_df['Error_Type'] = 'False Positive'

# Add prediction scores for analysis
false_positives_df['Prediction_Score'] = (X_dev[fp_indices] @ w_avg + b_avg)

print(f"\n{'='*70}")
print(f"FALSE POSITIVES: Predicted PASS, Actually FAILED ({num_fp_to_extract} examples)")
print(f"{'='*70}\n")

# Display with key features
display_cols = ['Gender', 'Age', 'Department', 'Attendance (%)', 'Midterm_Score', 
                'Final_Score', 'Assignments_Avg', 'Total_Score', 
                'Prediction_Score', 'Predicted_Grade', 'Actual_Grade']

print(false_positives_df[display_cols].to_string(index=True))

#  stats
print(f"\n{'='*70}")
print("False Positive Summary Statistics:")
print(f"{'='*70}")
print(f"Average Total Score: {false_positives_df['Total_Score'].mean():.2f}")
print(f"Average Final Score: {false_positives_df['Final_Score'].mean():.2f}")
print(f"Average Attendance: {false_positives_df['Attendance (%)'].mean():.2f}%")
print(f"Average Prediction Score: {false_positives_df['Prediction_Score'].mean():.4f}")


FALSE POSITIVES: Predicted PASS, Actually FAILED (15 examples)

     Gender  Age   Department  Attendance (%)  Midterm_Score  Final_Score  Assignments_Avg  Total_Score  Prediction_Score  Predicted_Grade  Actual_Grade
8         0   24           CS           70.32          41.14        83.66            66.76      65.4480     311376.803550                1             0
19        1   23  Engineering           52.96          78.38        55.22            60.21      67.2125     279356.486499                1             0
31        1   19           CS           66.87          72.25        88.65            54.94      69.7470     289219.908868                1             0
37        1   20  Engineering           65.60          76.57        75.55            54.99      61.2315     227794.060081                1             0
57        1   22  Mathematics           61.57          76.65        58.67            56.18      68.5950     273010.801482                1             0
75        0   22 

In [83]:
# Extract 10 False Negatives
num_fn_to_extract = min(10, len(false_negatives_indices))
fn_indices = false_negatives_indices[:num_fn_to_extract]

# Create DataFrame with all relevant information
false_negatives_df = dev_df.iloc[fn_indices].copy()
false_negatives_df['Predicted_Grade'] = 0  # Fail
false_negatives_df['Actual_Grade'] = 1     # Pass
false_negatives_df['Error_Type'] = 'False Negative'

# Add prediction scores for analysis
false_negatives_df['Prediction_Score'] = (X_dev[fn_indices] @ w_avg + b_avg)

print(f"\n{'='*70}")
print(f"FALSE NEGATIVES: Predicted FAIL, Actually PASSED ({num_fn_to_extract} examples)")
print(f"{'='*70}\n")

# Display with key features
print(false_negatives_df[display_cols].to_string(index=True))

# Summary statistics
print(f"\n{'='*70}")
print("False Negative Summary Statistics:")
print(f"{'='*70}")
print(f"Average Total Score: {false_negatives_df['Total_Score'].mean():.2f}")
print(f"Average Final Score: {false_negatives_df['Final_Score'].mean():.2f}")
print(f"Average Attendance: {false_negatives_df['Attendance (%)'].mean():.2f}%")
print(f"Average Prediction Score: {false_negatives_df['Prediction_Score'].mean():.4f}")


FALSE NEGATIVES: Predicted FAIL, Actually PASSED (10 examples)

    Gender  Age   Department  Attendance (%)  Midterm_Score  Final_Score  Assignments_Avg  Total_Score  Prediction_Score  Predicted_Grade  Actual_Grade
0        1   21           CS           92.88          45.43        51.62            94.52      70.5845      42801.751265                0             1
2        1   21     Business           98.22          47.15        85.39            97.60      72.3400     163187.006902                0             1
3        1   22     Business           80.57          99.83        87.61            90.90      82.2960     169692.011112                0             1
4        1   23  Mathematics           81.60          44.80        75.08            91.32      76.7290     249189.706768                0             1
5        0   24           CS           84.67          66.00        71.99            95.98      77.2365     176139.783562                0             1
6        1   21        

In [1]:
print("="*80)
print("SUMMARY")
print("="*80)
print()
print(f"Averaged Perceptron (trained with {n_perceptrons} models)")
print(f"Features used: {len(features_lst)}")
print(f"Dev set accuracy: {accuracy:.2f}%")
print()
print(f"Misclassification Analysis:")
print(f"  Total False Positives (Predicted PASS, Actual FAIL): {len(false_positives_indices)}")
print(f"  Total False Negatives (Predicted FAIL, Actual PASS): {len(false_negatives_indices)}")
print()

SUMMARY



NameError: name 'n_perceptrons' is not defined